In [34]:
import os, pathlib, time
from io import BytesIO
from PIL import Image, ImageDraw

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
from dotenv import load_dotenv
import openai
import os

In [112]:
from dotenv import load_dotenv
import openai
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

COMPUTER_VISION_ENDPOINT = os.getenv("COMPUTER_VISION_ENDPOINT")
COMPUTER_VISION_KEY = os.getenv("COMPUTER_VISION_KEY")


# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [ ]:
print(OPENAI_API_KEY)
print(COMPUTER_VISION_ENDPOINT)
print(COMPUTER_VISION_KEY)

In [36]:
key = COMPUTER_VISION_KEY
endpoint = COMPUTER_VISION_ENDPOINT

In [106]:
client = ComputerVisionClient(endpoint=endpoint, credentials=CognitiveServicesCredentials(key))

In [102]:
images_list = []
cropped_images_paths = []
cropped_images_list = []
working_directory = pathlib.Path ("./")
os.makedirs(os.path.join ("./", 'cropped_images'), exist_ok=True)
print (working_directory)

.


In [103]:
# Create an Image object from each image in a the Images folder.
for image_path in working_directory.glob('images/*.jpg'):  # assume all images are jpg
    imageObject = Image.open(image_path)
    images_list.append(imageObject)
    cropped_images_paths.append(pathlib.Path (str(image_path).replace('images', 'cropped_images')))

# Optional, draw bounding box around desired line of text, show 

In [83]:
print(f"Images to be cropped:{cropped_images_paths}")

Images to be cropped:[PosixPath('cropped_images/IMG_0217.jpg'), PosixPath('cropped_images/IMG_0211.jpg'), PosixPath('cropped_images/IMG_0212.jpg'), PosixPath('cropped_images/IMG_0217.jpg'), PosixPath('cropped_images/IMG_0211.jpg'), PosixPath('cropped_images/IMG_0212.jpg')]


In [104]:
# Crop each image in your list at the same place
for image in images_list:
    # Don't exceed your image height and width
    # w, h = image.size
    # print('Image width & height:', w, h)

    cropped = image.crop((150,100,1500,1500)) # edges: left, top, right, bottom
    cropped_images_list.append(cropped)

In [105]:
for i in range(len(cropped_images_list)):
    # Convert cropped images back to PIL.JpegImagePlugin.JpegImageFile type
    b = BytesIO()
    cropped_images_list[i].save(b, format="jpeg")
    cropped_images_list[i] = Image.open(b)
    # Save cropped image to file.
    cropped_images_list[i].save(cropped_images_paths[i], format="jpeg")    
    b.close()

In [107]:
for cropped_image_path in cropped_images_paths:
    cropped_bytes = open(cropped_image_path, "rb")
    # Call API
    result = client.recognize_printed_text_in_stream(cropped_bytes)

In [111]:
for region in result.regions:
        for line in region.lines:
            for word in line.words:
                #print("Extracted text:")
                print(word.text)
                #print()
                #print('Bounding box for each word:')
                #print(word.bounding_box)
print()

Server:
Shannon
At.
Description
THE
DORIAN
BURGER
TRUFFLE
FRIES
ADD
BACON
CHICKEN
SLIDERS
THE
DORIAN
BURGER
COKE
THE
DORIAN
BURGER
SUB
CAESAR
SALAD
CAESAR
SALAD
Total
23
$
3.50*
$
22.50*
$
16.00*
02:49
PM
The
Dorian
2001
Chestnut
St
12/03/2021
23-4
San
Francisco,
CA
94123
doriansf
.
com
12/03/2021
02:49
PM
The
Dorian
2001
Chestnut
St
San
Francisco,
CA
94123
314
Guests:
I
Amount
$
25.00*
$
15.00*
$
17.50*
$
99.50
4307
119097
Chip
TVR
Merchant
ID
Invoice
#
TSI
AID
doriansf
.com
314
Shannon
At.
8000008000
000000029082
2934503181
6800
A0000000041010
$
99.50
Subtotal
Health
Ordinance
$
5.48
K)
sales
Tax
Check
Total
Split
Payment
$
9.07
$
114.05
*(5.5%)
SF
Health
Ordinance
$
5.48
$
9.07
Sales
Tax
Grand
Total
$
114105
Book
your
social
or
corporate
event
with
us!
Email
Jazlyn@doriansf
for
more
info
Tip
Total
x
$3
IGIN



In [116]:
system_template = '''You are company financial department assistant. Your main task is to summarize the receipts.
The summary must contain all common information required for expense reimbursement.
More specifically is should contain the following information:
1. Date of the receipt
2. Vendor derails: name, address, phone number, all other information that can be useful
3. Transaction details: amount, currency, payment method, last 4 digits of the card, date of the transaction, transaction ID, service description
If some of the information is missing, please indicate that it is missing.
Output result as python json object with the fields listed above.
'''

text="Airport Travel Agency PO Box 251567, International Terminal -- San Francisco International Airport San Francisco, CA 94125 (650) 877-0422 PURCHASE Authorization 348413 Receipt 50WW Mastercard AID AO 00 00 00 04 10 10 Small Small Total MasterCard 4307 (Chip) Vladislav Feigin Dec 3, 2021 8:24 AM S20.oo S20.oo $20.00 SAVE $5 OFF NAP ROOMS & SHOWERS AT FRESHEN UP WHEN YOU STORE BAGS AT AIRPORT TRAVEL AGENCY Return Policy: NO REFUNDS FOR BAGGAGE STORAGE"

In [121]:
# prepare prompt
messages = [{"role": "system", "content": f"{system_template}"},
            {"role": "user", "content": f"{text}"}]


answer = openai.ChatCompletion.create(engine=OPENAI_DEPLOYMENT_NAME,

                                      messages=messages,)

print(answer.choices[0].message.content)

{
   "Date":"Dec 3, 2021 8:24 AM",
   "Vendor":{
      "Name":"Airport Travel Agency",
      "Address":"PO Box 251567, International Terminal -- San Francisco International Airport San Francisco, CA 94125",
      "Phone":"(650) 877-0422"
   },
   "Transaction":{
      "Amount":"$20.00",
      "Currency":"USD",
      "Payment Method":"Mastercard",
      "Last 4 Digits of Card":"4307",
      "Transaction ID":"348413",
      "Service Description":"SAVE $5 OFF NAP ROOMS & SHOWERS AT FRESHEN UP WHEN YOU STORE BAGS AT AIRPORT TRAVEL AGENCY"
   }
}
